In [12]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "myTester"
password = "Panda367"
shelter = AnimalShelter(username, password)

# Initial load (all records)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare logo setup
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'200px'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H1("Project Two - Daniel Beale", style={'textAlign': 'center', 'color': '#2c3e50'}),
    html.Hr(),

    # ---- Radio Button Filters ----
    html.Div([
        html.Label("Select Rescue Type:", style={'font-weight': 'bold'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'}
            ],
            value='all',
            inline=True
        )
    ], style={'textAlign': 'center'}),
    
    html.Hr(),

    # ---- Data Table ----
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'maxWidth': '200px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230,230,230)',
            'fontWeight': 'bold'
        }
    ),
    html.Br(),
    html.Hr(),

    # ---- Chart + Map Side by Side ----
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# ---- MongoDB Queries for Filters ----
def get_query(filter_type):
    if filter_type == 'water':
        return {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'mountain':
        return {
            "$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'disaster':
        return {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        return {}  # Default: show all dogs


# ---- Update Data Table ----
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = get_query(filter_type)
    filtered_df = pd.DataFrame(list(db.read(query)))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')


# ---- DataTable Highlight Style ----
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]


# ---- Breed Distribution Chart ----
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graph(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.H5("No data available for this filter.")]
    
    fig = px.bar(dff, x="breed", title="Breed Distribution for Selected Rescue Type",
                 labels={"breed": "Breed", "count": "Number of Dogs"})
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    return [dcc.Graph(figure=fig)]


# ---- Geolocation Map ----
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.H5("No location data to display.")]
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9])
                                 ])
                             ])
               ])
    ]


# ---- Run App ----
app.run_server()

Dash app running on https://recycleambient-latinbamboo-3000.codio.io/proxy/8050/
